In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-08-27 00:32:45.677881


In [5]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [7]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [9]:
current_date.to_datetime64()

numpy.datetime64('2025-08-27T00:00:00.000000000')

In [11]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [13]:
fetch_data_to

Timestamp('2025-08-27 00:00:00+0000', tz='UTC')

In [15]:
fetch_data_from

Timestamp('2025-07-29 00:00:00+0000', tz='UTC')

In [17]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)
    # - timedelta(weeks=52)

    print(historical_from_date)
    print(historical_to_date)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date.to_datetime64()]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date.to_datetime64()]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)
    print(rides['pickup_datetime'])

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [19]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

2024-07-30 00:00:00+00:00
2024-08-28 00:00:00+00:00
File already exists for 2024-01.
Loading data for 2024-01...
Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,514,289
Valid records: 3,450,929
Records dropped: 63,360 (1.80%)
Successfully processed data for 2024-04.
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 3,723,833
Valid records: 3,653,829
Records dropped: 70,004 (1.88%)
Successfully processed data for 2024-05.
File already exists for 2024

In [20]:
rides

,pickup_datetime,pickup_location_id
1001595,2025-08-08 17:38:14,2
18368211,2025-08-08 17:38:14,2
1914230,2025-08-19 19:22:03,2
19280846,2025-08-19 19:22:03,2
2194440,2025-08-23 00:27:11,2
...,...,...
17248932,2025-12-30 23:52:35,263
17247696,2025-12-30 23:53:15,263
17247067,2025-12-30 23:55:24,263
17248597,2025-12-30 23:56:59,263


In [23]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [25]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-07-29 00:00:00,2,0
1,2025-07-29 01:00:00,2,0
2,2025-07-29 02:00:00,2,0
3,2025-07-29 03:00:00,2,0
4,2025-07-29 04:00:00,2,0
...,...,...,...
967195,2025-12-30 19:00:00,263,148
967196,2025-12-30 20:00:00,263,231
967197,2025-12-30 21:00:00,263,247
967198,2025-12-30 22:00:00,263,193


In [27]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967200 entries, 0 to 967199
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         967200 non-null  datetime64[ns]
 1   pickup_location_id  967200 non-null  int16         
 2   rides               967200 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 11.1 MB


In [29]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-08-26 20:34:02,747 INFO: Initializing external client
2025-08-26 20:34:02,747 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-08-26 20:34:04,947 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1243294


In [31]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█| Rows 967200/967200 | Elapsed Time: 00:24 | Rema


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1243294/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)